# Setup 


In [9]:
import os
from openai import OpenAI

import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import BaseOutputParser
import json
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:

client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

completion = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    messages=[
        {
            "role": "user",
            "content": """Hello!"""
        }
    ],
    temperature=0.6
)

print(completion.to_json())


{
  "id": "chatcmpl-5201472be1a3433cbcf14a68c96e3330",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "audio": null,
        "function_call": null,
        "tool_calls": [],
        "reasoning_content": null
      },
      "stop_reason": null
    }
  ],
  "created": 1748194898,
  "model": "meta-llama/Meta-Llama-3.1-70B-Instruct",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 12,
    "total_tokens": 22,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "prompt_logprobs": null
}


In [7]:
llm = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    temperature=0.6
)

In [ ]:
# Create a simple prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("assistant", "How can I assist you today?"),
    ("human", "{question}")
])

# Create a simple chain
chain = prompt | llm

# Test it
response = chain.invoke({"question": "Hello!"})
print(response.content)

Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?


# CP 1. Climate legislation Pipeline

##### Indicator CP 1.a Does the country have a framework climate law or equivalent?

Individualy test the following (A country is assessed as ‘Yes’ if it has a framework climate law that fulfils all of
the following):

1. It sets a strategic direction for decarbonisation (i.e. it must include a clear
statement to meet the goals of the Paris Agreement or a national long-term
decarbonisation target)

2. It is enshrined in law (i.e. it must be legislative rather than executive, except in
particular political systems)

3. It sets out at least one of the following obligations: meeting a national target;
developing, revising, implementing or complying with domestic plans,
strategies or policies; developing policy instruments such as regulation, taxation
or public spending in support of climate change goals

4. Also check this In exceptional cases, the combination of a broad environmental law and a clearly
linked executive climate strategy may be sufficient to meet these criteria

**Then have other model check if 1,2,3 are satisfied or 4 is satisfied -> answer yes**

**Then have another model format output into markdown using (this)[https://python.langchain.com/docs/tutorials/extraction/]**



### 1.1 Chain of thought Implementation
#### 1.1.1 Prompting

In [10]:
# Define individual criterion evaluation prompts
criterion_1_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation. 
    Your task is to evaluate whether a climate law sets a strategic direction for decarbonisation.
    
    A law meets this criterion if it includes a clear statement to meet the goals of the Paris Agreement 
    OR a national long-term decarbonisation target.
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this law set a strategic direction for decarbonisation?")
])

criterion_2_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a climate law is enshrined in law.
    
    A law meets this criterion if it is legislative rather than executive 
    (except in particular political systems where executive action has legal force).
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nIs this law enshrined in law (legislative rather than executive)?")
])

criterion_3_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate whether a climate law sets out specific obligations.
    
    A law meets this criterion if it sets out at least one of the following:
    - Meeting a national target
    - Developing, revising, implementing or complying with domestic plans, strategies or policies
    - Developing policy instruments such as regulation, taxation or public spending in support of climate goals
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this law set out the required obligations?")
])

criterion_4_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation.
    Your task is to evaluate the exceptional case criterion.
    
    This criterion is met if there is a combination of:
    - A broad environmental law AND
    - A clearly linked executive climate strategy
    
    This combination may be sufficient to meet the framework criteria in exceptional cases.
    
    Respond with only 'YES' or 'NO' followed by a brief explanation."""),
    ("human", "Context: {context}\n\nDoes this represent a valid exceptional case (broad environmental law + linked executive strategy)?")
])

# Final assessment prompt
final_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst making a final assessment of climate legislation.
    
    A country is assessed as 'YES' for having framework climate law if:
    - Criteria 1, 2, AND 3 are all satisfied, OR
    - Criterion 4 is satisfied (exceptional case)
    
    Based on the individual assessments, provide a final 'YES' or 'NO' answer with reasoning."""),
    ("human", """Individual criterion assessments:
    Criterion 1 (Strategic direction): {criterion_1_result}
    Criterion 2 (Enshrined in law): {criterion_2_result}
    Criterion 3 (Obligations): {criterion_3_result}
    Criterion 4 (Exceptional case): {criterion_4_result}
    
    What is the final assessment?""")
])

#### 1.1.2 Retrieve Context


#### 1.1.3 Run

In [ ]:
# Create individual chains
criterion_1_chain = criterion_1_prompt | llm
criterion_2_chain = criterion_2_prompt | llm
criterion_3_chain = criterion_3_prompt | llm
criterion_4_chain = criterion_4_prompt | llm

# Create a function to run all criterion evaluations
def evaluate_all_criteria(inputs):
    """Evaluate all four criteria and return results for final assessment"""
    criterion_1_result = criterion_1_chain.invoke({"context": inputs["context_1"]})
    criterion_2_result = criterion_2_chain.invoke({"context": inputs["context_2"]})
    criterion_3_result = criterion_3_chain.invoke({"context": inputs["context_3"]})
    criterion_4_result = criterion_4_chain.invoke({"context": inputs["context_4"]})
    
    return {
        "criterion_1_result": criterion_1_result.content,
        "criterion_2_result": criterion_2_result.content,
        "criterion_3_result": criterion_3_result.content,
        "criterion_4_result": criterion_4_result.content
    }

# Create the complete chained pipeline
from langchain.schema.runnable import RunnableLambda

complete_assessment_chain = (
    RunnableLambda(evaluate_all_criteria)
    | final_assessment_prompt 
    | llm
)

# Test the complete chain
test_contexts = {
    "context_1": "Sample climate law that mentions Paris Agreement goals and net-zero by 2050",
    "context_2": "This is a legislative act passed by parliament, not an executive order",
    "context_3": "The law requires development of national climate plans and carbon pricing mechanisms",
    "context_4": "No broad environmental law with executive strategy present"
}

Final Assessment:
Based on the individual assessments, I conclude that the country has a framework climate law: **YES**

The reason is that all three main criteria (1, 2, and 3) are satisfied. The law provides a clear strategic direction, is enshrined in law, and imposes obligations on the country to develop national climate plans and carbon pricing mechanisms. Since Criterion 4 is not satisfied, it is not an exceptional case, and we rely on the satisfaction of the first three criteria.


In [ ]:
# Run the complete pipeline
result = complete_assessment_chain.invoke(test_contexts)
#some function that takes indicator, retrieves context and then runs complete_assessment_chain fucntion
print("Final Assessment:")
print(result.content)

#### 1.2 Alternative Large context window Gemini Implementation

In [ ]:
# Load larger context LLM
large_context_llm = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Llama-3.3-70B-Instruct",
    temperature=0.6
)

In [ ]:
# Set context to whole entire corpus of relevant documetns according to slyvan sheet

all_documents_context =

In [ ]:
# Single comprehensive climate legislation assessment prompt
comprehensive_assessment_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert legal analyst specializing in climate legislation assessment. 

Your task is to evaluate whether a country has a framework climate law based on specific criteria and provide a structured markdown assessment.

EVALUATION CRITERIA:
A country is assessed as 'YES' if it has a framework climate law that fulfils ALL of criteria 1, 2, AND 3, OR criterion 4:

1. STRATEGIC DIRECTION: Sets a strategic direction for decarbonisation (must include a clear statement to meet the goals of the Paris Agreement OR a national long-term decarbonisation target)

2. ENSHRINED IN LAW: Is enshrined in law (must be legislative rather than executive, except in particular political systems)

3. OBLIGATIONS: Sets out at least one of the following obligations:
   - Meeting a national target
   - Developing, revising, implementing or complying with domestic plans, strategies or policies
   - Developing policy instruments such as regulation, taxation or public spending in support of climate goals

4. EXCEPTIONAL CASE: The combination of a broad environmental law AND a clearly linked executive climate strategy may be sufficient to meet these criteria

ASSESSMENT LOGIC:
- If criteria 1, 2, AND 3 are all satisfied → YES
- If criterion 4 is satisfied → YES
- Otherwise → NO

OUTPUT FORMAT:
Provide your assessment in the following markdown format:

```markdown
# Climate Legislation Assessment: CP 1.a Framework Climate Law

## Individual Criterion Evaluation

### Criterion 1: Strategic Direction for Decarbonisation
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of whether the law includes clear Paris Agreement goals or long-term decarbonisation targets]

### Criterion 2: Enshrined in Law
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of whether this is legislative rather than executive]

### Criterion 3: Sets Out Obligations
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of which obligations are present, if any]

### Criterion 4: Exceptional Case
**Result:** [YES/NO]
**Reasoning:** [Brief explanation of whether broad environmental law + executive strategy combination exists]

## Final Assessment

**Overall Result:** [YES/NO]

**Logic Applied:** [Explain whether criteria 1+2+3 are satisfied OR criterion 4 is satisfied]

**Conclusion:** [Brief summary of why the country does/does not have a framework climate law]
```"""),
    ("human", "Context: {context}\n\nPlease evaluate whether this country has a framework climate law based on the provided context.")
])

# Create the single-model chain
single_model_assessment_chain = comprehensive_assessment_prompt | large_context_llm


# Run the assessment
result = single_model_assessment_chain.invoke({"context": all_documents_context})
print(result.content)